In [ ]:
# import ir_datasets
# dataset = ir_datasets.load("msmarco-passage")
# for doc in dataset.docs_iter():
#     doc # namedtuple<doc_id, text>

In [22]:
from elasticsearch import Elasticsearch
import pandas as pd
from sentence_transformers import CrossEncoder

In [18]:
es = Elasticsearch()
# es.info()
INDEX_NAME = "passage_index"
doc = es.get(index=INDEX_NAME, id=1)
print(doc)

{'_index': 'passage_index', '_type': '_doc', '_id': '1', '_version': 1, '_seq_no': 1, '_primary_term': 1, 'found': True, '_source': {'content': 'The Manhattan Project and its atomic bomb helped bring an end to World War II. Its legacy of peaceful uses of atomic energy continues to have an impact on history and science.'}}


In [42]:
queries_eval = pd.read_csv("data/queries/queries.eval.tsv", sep='\t', header=None)#, index_col=0)
queries_eval.head()

,0,1
0,786436,what is prescribed to treat thyroid storm
1,9,Refer to the data. Diminishing returns begin ...
2,786450,what is presentation software?
3,524308,treasury routing number
4,33,game called poem who wrote what occasion


In [46]:
# collectionFile = "data/collection/collection.tsv"
collection_df = pd.read_csv("data/collection/collection.tsv", sep='\t', header=None) #, index_col=0)
print(len(collection_df))
collection_df.head()

8841823


,0,1
0,0,The presence of communication amid scientific ...
1,1,The Manhattan Project and its atomic bomb help...
2,2,Essay on The Manhattan Project - The Manhattan...
3,3,The Manhattan Project was the name for a proje...
4,4,versions of each volume as well as complementa...


In [50]:
qrelspath = "data/queries/qrels.txt"

qrels = set()
with open(qrelspath, encoding="utf-8") as file:
    for line in file:
        l = line.split(' ')
        qrels.add(l[0])

In [ ]:
# def relevant_queries(queries, qrels):
#     relevant_queries = []
#     relevant_queries_id = []

#     for idx, query in enumerate(queries):
#         query_id = str(queries_id[idx])
#         if query_id in qrels:
#             relevant_queries.append(query)
#             relevant_queries_id.append(query_id)

#     return relevant_queries, relevant_queries_id

In [53]:
queriesToUse = {}
for _, query in queries_eval.iterrows():
    if str(query[0]) in qrels:
        queriesToUse[query[0]] = query[1]

In [56]:
model = CrossEncoder('cross-encoder/ms-marco-TinyBERT-L-2-v2', max_length=512)

ranked = {}

# Rank the top 1000 passages for each query
for query_id, query_text in queriesToUse.items():
    # print(query_id, query_text)
    # break
    for _, doc in collection_df.iterrows():
        # print(doc[0], doc[1])
        # break
        doc_id = doc[0]
        doc_text = doc[1]
        score = model.predict([(query_text, doc_text)])[0]
        if query_id not in ranked:
            ranked[query_id] = []
        ranked[query_id].append((doc_id, score))
    break

In [4]:
# from sentence_transformers import CrossEncoder
# model = CrossEncoder('cross-encoder/ms-marco-TinyBERT-L-2-v2', max_length=512)

# ranked = {}

# # Rank the top 1000 passages for each query
# for query in dataset.queries_iter():
#     query_id = query.query_id
#     query_text = query.query_text
#     for doc in dataset.docs_iter():
#         doc_id = doc.doc_id
#         doc_text = doc.text
#         score = model.predict([(query_text, doc_text)])[0]
#         if query_id not in ranked:
#             ranked[query_id] = []
#         ranked[query_id].append((doc_id, score))
#     ranked[query_id] = sorted(ranked[query_id], key=lambda x: x[1], reverse=True)[:1000]

array([-11.0084095, -11.049721 , -11.087914 ], dtype=float32)